In [ ]:
"""
Встраивание информации в изображение методом LSB.
"""

In [ ]:
def to_bin(s):
    return ''.join(format(x,'08b') for x in bytearray(s,'utf-8'))
def unbin(s):
    data = bytes([int(s[i:i+8],2) for i in range(0,len(s),8)]).decode('utf-8')
    return data

msg = "The tower, which was not supposed to be there, plunges into the earth in a place \
just before the black pine forest begins to give way to swamp and then the reeds and \
wind-gnarled trees of the marsh flats."
msg_b = to_bin(msg)

from PIL import Image

img = Image.open("reindeer.bmp")
img.save("test_lsb.bmp")
img = Image.open("test_lsb.bmp")

def write(img,msg):
    len_msg = len(msg)
    w, h = img.size 
    pixels = img.load()
    j = 0
    for x in range(0,w):
        if j>len_msg:
            break
        for y in range(0,h):
            p = list(pixels[x,y])
            for i in range(3):
                if j<len_msg:
                    p[i] = p[i] & ~1 | int(msg[j],2)
                    j+=1
            pixels[x,y] = tuple(p)
    return img

def extract(img):
    w, h = img.size
    pixels = img.load()
    res = ''
    for x in range(0,w):
        for y in range(0,h):
            p = pixels[x,y]
            res+=''.join([str(b&1) for b in p])
    return res

res = write(img,msg_b)
res.save("test_lsb.bmp")
res.show()
s1 = unbin(extract(img)[:len(msg_b)])
print(s1)
img.close()

In [ ]:
'''Часть 2
1. PSNR
2. График PSNR
3. Оценка целесообразности'''
import cv2
img1 = cv2.imread('reindeer.bmp')
img2 = cv2.imread('test_lsb.bmp')
psnr = cv2.PSNR(img1,img2)
print("PSNR: {}".format(psnr))
messages = [
"bioluminescence","The tower, which was not","The tower, which was not supposed to be there, plunges",
"The tower, which was not supposed to be there, plunges into the earth in a place just before the black",
"The tower, which was not supposed to be there, plunges into the earth in a place just before the black pine\
forest begins to give way to swamp and then, ","The tower, which was not supposed to be there,plunges into the \
earth in a place just before the black pine forest begins to give way to swamp and then the reeds and \
wind-gnarled trees of the marsh flats. Beyond the","The tower, which was not supposed to be there, plunges into\
the earth in a place just before the black pine forest begins to give way to swamp and then the reeds and \
wind-gnarled trees of the marsh flats. Beyond the marsh flats and the natural canals lies the ocean and, a  \
little farther down the coast, a derelict lighthouse.","bioluminescence "*100]

psnr_values = []
i = 0
for msg in messages:
    img = Image.open("reindeer.bmp")
    res = write(img,to_bin(msg))
    res.save(str(i)+img.filename)
    img2 = cv2.imread(str(i)+img.filename)
    psnr_values.append(cv2.PSNR(img1,img2))
    i+=1
    img.close()
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.patches as patches

large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': small,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
x = [len(msg.split(' ')) for msg in messages]
y = psnr_values
plt.figure(figsize=(16, 10), dpi= 80, facecolor='w', edgecolor='k')
ax = plt.gca()
ax.set(xlabel='Message length (words)', ylabel='PSNR value')
plt.plot(x,y,c='tab:green')
plt.title("PSNR vs length", fontsize=22)
plt.xticks(x)
plt.show()
#plt.savefig("stats.pdf",bbox_inches='tight')

In [ ]:
'''3. Визуальная атака'''
msg = "The tower, which was not supposed to be there, plunges into the earth in a place \
just before the black pine forest begins to give way to swamp and then the reeds and \
wind-gnarled trees of the marsh flats."*500

img4 = Image.open("reindeer.bmp")
msg_b = to_bin(msg)
print("Длина сообщения: {} бит".format(len(msg_b)))
write(img4,msg_b).save("visual_"+img4.filename)

def visual(img):
    h, w = img.size
    pixels = img.load()
    new_img = Image.new("RGB", (h, w), color=(0, 0, 0))
    for i in range(h):
        for j in range(w):
            p = pixels[i, j]
            if len(p) == 4:
                p = p[:-1]
            new_p = [0, 0, 0]
            for k in range(3):
                new_p[k] = 255 if int(bin(p[k])[-1]) == 1 else 0
            new_img.putpixel((i, j), tuple(new_p))
    new_img.save("LSB_" + img.filename)
visual(img4)